### 01. 모듈 import / AP몰 접속 / 오늘 하루 다시보지 않기 / brand name 검색

In [ ]:
########## 모듈 import ##########
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver as wd
import time
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException, ElementClickInterceptedException

driver = wd.Chrome(executable_path='/Users/hyunginchoi/Desktop/com/python/chromedriver')



########## url과 brand 설정 ##########
ap_url = 'https://www.amorepacificmall.com'
brand_name = '이니스프리'



########## url 접속 후 brand name 검색 ##########
driver.get(ap_url)

driver.find_element_by_class_name('layer_close').click()

driver.find_element_by_id('query_str').send_keys(brand_name)
driver.find_element_by_class_name('btn_search').click()
time.sleep(1)

### 02. 제품 ID 리스트에 저장 / 페이지 이동 반복 / 페이지가 끝나면 except

In [ ]:
obj_id = []

try:
    while True:
        ########## 각 제품의 id get ##########
        prd_cell = driver.find_elements_by_class_name('item.item-apply')
        for prd in prd_cell:
            prd_review_count = prd.find_element_by_class_name('user_atc').find_element_by_css_selector('span').get_attribute('class')[-1]
            ########## 리뷰 개수가 0이 아닐 때에만 리스트에 append ##########
            if prd_review_count != '0':
                obj_id.append(prd.find_element_by_css_selector('a').get_attribute('href')[-5:])
        
        ########## 다음 페이지로 이동 ##########
        driver.find_element_by_css_selector('span + a').click()
        time.sleep(0.5)

########## 다음 페이지가 없을 때 발생하는 에러 ##########
except NoSuchElementException as e:
    print('더이상 페이지가 없습니다')

### 03. 데이터 저장할 text file 생성

In [ ]:
prd_info = open("apmall_prd_info.txt", "w")
prd_reviews = open("apmall_prd_reviews.txt", "w")

### 04. 각 제품 url에 접속 / 정보 크롤링 / write

In [ ]:
k = 1
for i in obj_id:
    
    ########## 얻은 제품 id로 각 제품 url에 접속 ##########
    driver.get(ap_url + '/kr/ko/product/detail?onlineProdSn=' + i)
    time.sleep(0.5)
    
    ########## name, price, review count, point ##########
    prd_name = driver.find_element_by_class_name('product_name').text
    prd_price = driver.find_element_by_class_name('product_price').find_element_by_class_name('num').text
    review_count = driver.find_element_by_class_name('review.visibleReview').find_element_by_class_name('txt').text[3:]
    point = driver.find_element_by_css_selector('div > div.detail_head > div > div.product_info > div.purchase_benefit > p').text
    
    shipping_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dt > button').click()
    time.sleep(0.1)
    shipping = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(1) > dd > div > p').text
    
    purchase_benefit_btn = driver.find_element_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dt > button').click()
    time.sleep(0.1)
    purchase_benefits = driver.find_elements_by_css_selector('#ap_container > div > div.detail_head > div > div.product_info > div.purchase_benefit > div > dl:nth-child(2) > dd > div > ul > li')
    benefit_total = ''
    for benefit in purchase_benefits:
        span = benefit.find_element_by_css_selector('span').text
        em = '(' + benefit.find_element_by_css_selector('em').text + ')'
        benefit_total += span + em + " "
   

    ########## 리뷰 200개 이상이면 200으로 제한 ##########
    if ',' in review_count or int(review_count) > 200:
        review_count = '200'

    else:
        ########## 리뷰 200개가 보일 때까지 '더보기' 클릭 ##########
        review_click = driver.find_element_by_css_selector('#ap_container > div > div.detail_body.ui_tab.＠tabs-apply > div.tab_menu > ul > li:nth-child(2) > button').click()
        try:
            if int(review_count) < 200:
                while True:
                    driver.find_element_by_class_name('btn_list_more').click()
                    time.sleep(1)
            else:
                ########## 200개가 넘으면 더보기 버튼을 19번만 클릭 ##########
                num = 0
                while num < 20:
                    driver.find_element_by_class_name('btn_list_more').click()
                    time.sleep(1)
                    num += 1
        except:
            pass


        
        ########## date, rating, user_id, text ##########
        review_box = driver.find_elements_by_class_name('review_box')
        j = 1
        total_rating = 0
        for review in review_box:
            date = review.find_element_by_class_name('rating').find_element_by_class_name('date').text
            rating = review.find_element_by_class_name('rating').find_element_by_css_selector('span:nth-child(1)').get_attribute('class')[-1]
            user_id = review.find_element_by_class_name('name').text
            text = review.find_element_by_class_name('ellipsis.line5').text
            total_rating += int(rating)
            prd_reviews.write(str(k) + " | " + str(j) + " | " + date + " | " + rating + " | " + user_id + " | " + text + "\n")
            j += 1
        prd_reviews.write("\n")    
        
        ########## 리뷰 평균을 소수 둘째점 자리까지만 구하기 ##########
        rating_avg = str("%.2f" % (total_rating / int(review_count)))


        prd_info.write(str(k) + " | " + prd_name + " | " + prd_price + " | " + review_count + " | " + rating_avg + " | " + point[:9] + " | " + shipping[9:28] + " | " + benefit_total + "\n")
        print(str(k) + " 끝")
    k += 1

### 05. driver 및 file close

In [ ]:
prd_info.close()
prd_reviews.close()
driver.close()